In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
from glob import glob
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
# from pandasql import sqldf
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 

In [ ]:
imexport = pd.read_csv('/content/drive/MyDrive/농산물예측/final_train_imexport.csv')
weather = pd.read_csv('/content/drive/MyDrive/농산물예측/final_train_weather.csv')
pummok = pd.read_csv('/content/drive/MyDrive/농산물예측/final_train_pummok.csv')
dosomae = pd.read_csv('/content/drive/MyDrive/농산물예측/final_train_dosomae.csv')

In [ ]:
# 품목의 번호 데이터타입 int로 바꿔주고
def make_concat(pummok, weather, dosomae, imexport):
  df = pd.concat([pummok, weather], axis=1)
  return df



In [ ]:
# 검증해보니 틀린거없다.

# df = pd.concat([pummok, weather], axis=1)
# tmp = []
# for i in range(len(df)):
#   if df['번호'][i]!= df['품목'][i]:
#     tmp.append(i)
# print(tmp)

[]


In [ ]:
imexport = pd.read_csv('/content/drive/MyDrive/농산물예측/eximport.csv')
weather = pd.read_csv('/content/drive/MyDrive/농산물예측/final_train_weather.csv')
pummok = pd.read_csv('/content/drive/MyDrive/농산물예측/final_train_pummok.csv')
dosomae = pd.read_csv('/content/drive/MyDrive/농산물예측/final_train_dosomae.csv')

In [ ]:
df = pd.concat([pummok, weather], axis=1)

# 품목, 날짜별로 merge를 위해 연월별
df['datadate'] = df['datadate'].astype('str')
# df['datadate'] = pd.to_datetime(df['datadate'])
df['yyyymm'] =  df['datadate'].apply(lambda x: x[:7] )
df
imexport['for_merge_tmp'] = imexport.apply(lambda x: str(x['datadate'])[:7]+'-'+str(x['num']), axis=1)

imexport.drop('datadate',axis=1, inplace=True)

df = pd.merge(df, imexport, how='left', left_on='tmp', right_on='for_merge_tmp')

# df.drop(['tmp', 'for_merge_tmp', '번호', 'num'], axis=1, inplace=True)

# df.fillna(0, inplace=True)

# -------------------------------------------

# 품목, 날짜별로 이너조인
# dosomae['datadate'] = dosomae['datadate'].astype('str')
# dosomae['datadate'] = dosomae['datadate'].apply(lambda x: str(x[:4])+'-'+str(x[4:6])+'-'+str(x[6:]))
# df['tmp1'] = df.apply(lambda x: str(x['datadate'])+str(x['품목']), axis=1)
# dosomae['tmp2'] = dosomae.apply(lambda x: str(x['datadate'])+str(x['품목']), axis=1)

# df2 = pd.merge(df, dosomae, how='left', left_on='tmp1', right_on='tmp2')

# # df2.drop(['tmp1', 'tmp2'], axis=1, inplace=True)

# df2


KeyError: ignored

In [ ]:
imexport = pd.read_csv('/content/drive/MyDrive/농산물예측/eximport.csv')
weather = pd.read_csv('/content/drive/MyDrive/농산물예측/final_train_weather.csv')
pummok = pd.read_csv('/content/drive/MyDrive/농산물예측/final_train_pummok.csv')
dosomae = pd.read_csv('/content/drive/MyDrive/농산물예측/final_train_dosomae.csv')

In [ ]:
df = pd.concat([pummok, weather], axis=1)

# 품목, 날짜별로 merge를 위해 연월별
df['datadate'] = df['datadate'].astype('str')
# df['datadate'] = pd.to_datetime(df['datadate'])
df['yyyymm'] =  df['datadate'].apply(lambda x: x[:7] )

imexport['for_merge_tmp'] = imexport.apply(lambda x: str(x['datadate'])[:7]+'-'+str(x['num']), axis=1)
df['번호'] = df['번호'].astype(int)
df['for_merge_tmp'] = df.apply(lambda x: str(x['datadate'])[:7]+'-'+str(x['번호']), axis=1)
imexport.drop('datadate',axis=1, inplace=True)

df = pd.merge(df,imexport, how='left',on='for_merge_tmp')
df
df.drop(['yyyymm', 'for_merge_tmp', '번호', 'num'], axis=1, inplace=True)

df.fillna(0, inplace=True)

# -------------------------------------------

# 품목, 날짜별로 이너조인
dosomae['datadate'] = dosomae['datadate'].astype('str')
dosomae['datadate'] = dosomae['datadate'].apply(lambda x: str(x[:4])+'-'+str(x[4:6])+'-'+str(x[6:]))
df['tmp1'] = df.apply(lambda x: str(x['datadate'])+str(x['품목']), axis=1)
dosomae['tmp2'] = dosomae.apply(lambda x: str(x['datadate'])+str(x['품목']), axis=1)

df2 = pd.merge(df, dosomae, how='left', left_on='tmp1', right_on='tmp2')

df2.drop(['tmp1', 'tmp2', 'datadate_y', '품목_y'], axis=1, inplace=True)

df2.rename(columns={'datadate_x':'datadate', '품목_x':'품목'}, inplace=True)

df2


,datadate,단가(원),거래량,해당일자_전체평균가격(원),주산지0_초기온도,주산지0_최대온도,주산지0_최저온도,주산지0_평균온도,주산지0_강수량(ml),주산지0_습도(%),...,수출금액(달러),수입중량(kg),수입금액(달러),무역수지(달러),일자별_도매가격_최대(원),일자별_도매가격_평균(원),일자별_도매가격_최소(원),일자별_소매가격_최대(원),일자별_소매가격_평균(원),일자별_소매가격_최소(원)
0,2013-01-01,0.000000,0.000000,0.000000,-3.9,4.4,-7.7,-1.4,0.0,0.0,...,0.0,1864728.0,1195211.0,-1195211.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,2013-01-02,16802.716667,7.333333,20712.240909,2.8,6.7,-1.1,3.5,0.0,0.0,...,0.0,1864728.0,1195211.0,-1195211.0,180000.0,178800.0,174000.0,5980.0,4298.522727,3210.0
2,2013-01-03,4454.533636,49.181818,3401.863678,-1.2,5.0,-6.3,-1.2,0.0,0.0,...,0.0,1864728.0,1195211.0,-1195211.0,180000.0,178800.0,174000.0,5980.0,4298.522727,3210.0
3,2013-01-04,5287.541667,46.500000,6553.903226,-1.7,6.8,-4.5,0.9,0.0,0.0,...,0.0,1864728.0,1195211.0,-1195211.0,180000.0,178800.0,174000.0,5980.0,4298.522727,3210.0
4,2013-01-05,6308.730000,72.529412,4299.079157,-4.6,6.2,-6.1,-0.7,0.0,0.0,...,0.0,1864728.0,1195211.0,-1195211.0,180000.0,178800.0,174000.0,5980.0,4298.522727,3210.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54052,2016-12-27,0.000000,0.000000,0.000000,-1.7,10.1,-2.8,3.1,0.0,72.3,...,1810615.0,0.0,0.0,1810615.0,0.0,0.0,0.0,0.0,0.000000,0.0
54053,2016-12-28,2753.411286,1009.285714,2428.960251,0.4,11.7,-1.4,5.3,0.0,73.1,...,1810615.0,0.0,0.0,1810615.0,14000.0,13180.0,12000.0,700.0,507.500000,370.0
54054,2016-12-29,2799.843465,993.881890,2512.875229,-1.5,3.9,-4.6,-0.1,0.0,88.1,...,1810615.0,0.0,0.0,1810615.0,14300.0,13860.0,13000.0,700.0,510.681818,370.0
54055,2016-12-30,3053.932411,1039.475177,2665.738013,-4.6,-4.6,-12.3,-8.9,0.0,55.1,...,1810615.0,0.0,0.0,1810615.0,15300.0,14860.0,14000.0,700.0,515.681818,370.0


In [ ]:
df2['datadate','일자별_도매가격_최대(원)', '일자별_도매가격_평균(원)', '일자별_도매가격_최소(원)', '일자별_소매가격_최대(원)', '일자별_소매가격_평균(원)', '일자별_소매가격_최소(원)']

KeyError: ignored

In [ ]:
df2.to_csv('real_real_final_concat.csv',index=False)

In [ ]:
imexport

,수출중량,수출금액,수입중량,수입금액,무역수지,num,for_merge_tmp
0,0.0,0.0,1864728.0,1195211.0,-1195211.0,0,2013-01-0
1,80.0,695.0,1285922.0,999313.0,-998618.0,0,2013-02-0
2,0.0,0.0,1489609.0,1464733.0,-1464733.0,0,2013-03-0
3,0.0,0.0,1441550.0,1190657.0,-1190657.0,0,2013-04-0
4,5.0,40.0,1212193.0,865478.0,-865438.0,0,2013-05-0
...,...,...,...,...,...,...,...
3745,26.0,78.0,0.0,0.0,78.0,36,2014-02-36
3746,31.0,85.0,0.0,0.0,85.0,36,2014-03-36
3747,20.0,98.0,0.0,0.0,98.0,36,2014-04-36
3748,82.0,304.0,0.0,0.0,304.0,36,2014-06-36


In [ ]:
df['for_merge_tmp'].nunique(), imexport['for_merge_tmp'].nunique()

(1776, 1344)

In [ ]:
imexport['for_merge_tmp'].nunique()

1344

In [ ]:
imexport['for_merge_tmp'] = imexport.apply(lambda x: str(x['datadate'])[:7]+'-'+str(x['num']), axis=1)
imexport

,datadate,수출중량,수출금액,수입중량,수입금액,무역수지,num,for_merge_tmp
0,2013-01-01,0.0,0.0,1864728.0,1195211.0,-1195211.0,0,2013-01-0
1,2013-02-01,80.0,695.0,1285922.0,999313.0,-998618.0,0,2013-02-0
2,2013-03-01,0.0,0.0,1489609.0,1464733.0,-1464733.0,0,2013-03-0
3,2013-04-01,0.0,0.0,1441550.0,1190657.0,-1190657.0,0,2013-04-0
4,2013-05-01,5.0,40.0,1212193.0,865478.0,-865438.0,0,2013-05-0
...,...,...,...,...,...,...,...,...
3745,2014-02-01,26.0,78.0,0.0,0.0,78.0,36,2014-02-36
3746,2014-03-01,31.0,85.0,0.0,0.0,85.0,36,2014-03-36
3747,2014-04-01,20.0,98.0,0.0,0.0,98.0,36,2014-04-36
3748,2014-06-01,82.0,304.0,0.0,0.0,304.0,36,2014-06-36


In [ ]:
imexport['num'].nunique()

28

In [ ]:
imexport['for_merge_tmp'] = imexport.apply(lambda x: str(x['datadate'])[:7]+str(x['num']), axis=1)

In [ ]:
pummok['번호'] = pummok['번호'].astype(int)
pummok['for_merge_tmp'] = pummok.apply(lambda x: str(x['datadate'])[:7]+'-'+str(x['번호']), axis=1)
pummok

,datadate,단가(원),거래량,해당일자_전체평균가격(원),번호,for_merge_tmp
0,2013-01-01,0.000000,0.000000,0.000000,0,2013-01-0
1,2013-01-02,16802.716667,7.333333,20712.240909,0,2013-01-0
2,2013-01-03,4454.533636,49.181818,3401.863678,0,2013-01-0
3,2013-01-04,5287.541667,46.500000,6553.903226,0,2013-01-0
4,2013-01-05,6308.730000,72.529412,4299.079157,0,2013-01-0
...,...,...,...,...,...,...
54052,2016-12-27,0.000000,0.000000,0.000000,36,2016-12-36
54053,2016-12-28,2753.411286,1009.285714,2428.960251,36,2016-12-36
54054,2016-12-29,2799.843465,993.881890,2512.875229,36,2016-12-36
54055,2016-12-30,3053.932411,1039.475177,2665.738013,36,2016-12-36


In [ ]:
weather

,주산지0_초기온도,주산지0_최대온도,주산지0_최저온도,주산지0_평균온도,주산지0_강수량(ml),주산지0_습도(%),주산지1_초기온도,주산지1_최대온도,주산지1_최저온도,주산지1_평균온도,주산지1_강수량(ml),주산지1_습도(%),주산지2_초기온도,주산지2_최대온도,주산지2_최저온도,주산지2_평균온도,주산지2_강수량(ml),주산지2_습도(%),품목
0,-3.9,4.4,-7.7,-1.4,0.0,0.0,0.9,9.0,-1.3,4.4,0.0,0.0,-2.3,6.0,-4.9,-0.3,0.0,95.4,0
1,2.8,6.7,-1.1,3.5,0.0,0.0,3.5,10.0,2.4,5.4,0.0,0.0,-1.8,7.7,-2.3,3.7,0.0,93.9,0
2,-1.2,5.0,-6.3,-1.2,0.0,0.0,3.3,8.3,-3.4,2.7,0.0,0.0,4.4,6.8,-3.6,1.5,0.0,82.1,0
3,-1.7,6.8,-4.5,0.9,0.0,0.0,-0.6,10.2,-1.0,5.8,0.0,0.0,-3.3,8.4,-4.3,1.9,0.0,88.4,0
4,-4.6,6.2,-6.1,-0.7,0.0,0.0,-0.6,8.4,-2.0,3.7,0.0,0.0,-4.2,8.4,-4.9,1.3,0.0,83.7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54052,-1.7,10.1,-2.8,3.1,0.0,72.3,-1.8,9.5,-3.0,3.2,0.0,67.1,5.7,8.5,1.0,6.5,7.9,0.0,36
54053,0.4,11.7,-1.4,5.3,0.0,73.1,1.5,11.3,1.0,5.3,0.0,69.9,1.0,12.7,1.0,7.1,11.0,0.0,36
54054,-1.5,3.9,-4.6,-0.1,0.0,88.1,2.4,7.1,1.0,3.4,0.0,76.5,1.6,8.6,1.0,4.5,2.8,0.0,36
54055,-4.6,-4.6,-12.3,-8.9,0.0,55.1,0.7,-1.0,-10.8,-6.9,0.0,28.8,0.9,-1.0,-4.1,-2.6,1.2,0.0,36


In [ ]:
len(pummok['datadate'].unique())

1461

In [ ]:
len(imexport['datadate'].unique())

48

In [ ]:
df['datadate'].dt.month

0         1
1         1
2         1
3         1
4         1
         ..
54052    12
54053    12
54054    12
54055    12
54056    12
Name: datadate, Length: 54057, dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 127295 entries, 0 to 127294
Data columns (total 32 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   datadate        127295 non-null  object 
 1   단가(원)           127295 non-null  float64
 2   거래량             127295 non-null  float64
 3   해당일자_전체평균가격(원)  127295 non-null  float64
 4   번호              127295 non-null  float64
 5   주산지0_초기온도       127295 non-null  float64
 6   주산지0_최대온도       127295 non-null  float64
 7   주산지0_최저온도       127295 non-null  float64
 8   주산지0_평균온도       127295 non-null  float64
 9   주산지0_강수량(ml)    127295 non-null  float64
 10  주산지0_습도(%)      127295 non-null  float64
 11  주산지1_초기온도       127295 non-null  float64
 12  주산지1_최대온도       127295 non-null  float64
 13  주산지1_최저온도       127295 non-null  float64
 14  주산지1_평균온도       127295 non-null  float64
 15  주산지1_강수량(ml)    127295 non-null  float64
 16  주산지1_습도(%)      127295 non-null  float64
 17  주산지2_초기온도 

In [ ]:
imexport.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3750 entries, 0 to 3749
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   datadate  3750 non-null   object 
 1   수출중량      3750 non-null   float64
 2   수출금액      3750 non-null   float64
 3   수입중량      3750 non-null   float64
 4   수입금액      3750 non-null   float64
 5   무역수지      3750 non-null   float64
 6   num       3750 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 205.2+ KB


In [ ]:
imexport['num'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  9, 10, 11, 12, 13, 15, 17, 18, 19, 20,
       21, 22, 25, 28, 29, 30, 32, 33, 34, 35, 36])

In [ ]:
imexport.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3750 entries, 0 to 3749
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   datadate       3750 non-null   object 
 1   수출중량           3750 non-null   float64
 2   수출금액           3750 non-null   float64
 3   수입중량           3750 non-null   float64
 4   수입금액           3750 non-null   float64
 5   무역수지           3750 non-null   float64
 6   num            3750 non-null   int64  
 7   for_merge_tmp  3750 non-null   object 
dtypes: float64(5), int64(1), object(2)
memory usage: 234.5+ KB


In [ ]:
df2.isna().sum()

datadate_x        0
단가(원)             0
거래량               0
해당일자_전체평균가격(원)    0
주산지0_초기온도         0
주산지0_최대온도         0
주산지0_최저온도         0
주산지0_평균온도         0
주산지0_강수량(ml)      0
주산지0_습도(%)        0
주산지1_초기온도         0
주산지1_최대온도         0
주산지1_최저온도         0
주산지1_평균온도         0
주산지1_강수량(ml)      0
주산지1_습도(%)        0
주산지2_초기온도         0
주산지2_최대온도         0
주산지2_최저온도         0
주산지2_평균온도         0
주산지2_강수량(ml)      0
주산지2_습도(%)        0
품목_x              0
수출중량              0
수출금액              0
수입중량              0
수입금액              0
무역수지              0
tmp1              0
datadate_y        0
품목_y              0
일자별_도매가격_최대(원)    0
일자별_도매가격_평균(원)    0
일자별_도매가격_최소(원)    0
일자별_소매가격_최대(원)    0
일자별_소매가격_평균(원)    0
일자별_소매가격_최소(원)    0
tmp2              0
dtype: int64

In [ ]:
dosomae['datadate'] = dosomae['datadate'].astype('str')
dosomae['datadate'][0][:4]

'2013'

In [ ]:
dosomae['datadate'][0][4:6]

'01'

In [ ]:
dosomae['datadate'][0][6:]

'01'

In [ ]:
dosomae

,datadate,품목,일자별_도매가격_최대(원),일자별_도매가격_평균(원),일자별_도매가격_최소(원),일자별_소매가격_최대(원),일자별_소매가격_평균(원),일자별_소매가격_최소(원),tmp2
0,2013-10-,3,0.0,0.0,0.0,0.0,0.000000,0.0,2013-10-3
1,2013-10-,3,43000.0,39200.0,35000.0,30000.0,18206.052632,9900.0,2013-10-3
2,2013-10-,3,43000.0,39200.0,35000.0,30000.0,18250.789474,9900.0,2013-10-3
3,2013-10-,3,43000.0,39200.0,35000.0,30000.0,18250.789474,9900.0,2013-10-3
4,2013-10-,3,43000.0,39600.0,35000.0,30000.0,18479.444444,12250.0,2013-10-3
...,...,...,...,...,...,...,...,...,...
54052,2016-22-,7,0.0,0.0,0.0,0.0,0.000000,0.0,2016-22-7
54053,2016-22-,7,47800.0,35960.0,31000.0,17980.0,12375.806452,8000.0,2016-22-7
54054,2016-22-,7,47800.0,37160.0,34000.0,17980.0,12375.806452,8000.0,2016-22-7
54055,2016-23-,7,47800.0,37160.0,34000.0,17980.0,12201.000000,8000.0,2016-23-7


In [ ]:
imexport['for_merge_tmp'] = imexport.apply(lambda x: x['datadate'][:7]+str(x['num']), axis=1)
imexport
#12*37*4

,datadate,수출중량,수출금액,수입중량,수입금액,무역수지,num,for_merge_tmp
0,2013-01-01,0.0,0.0,1864728.0,1195211.0,-1195211.0,0,2013-010
1,2013-02-01,80.0,695.0,1285922.0,999313.0,-998618.0,0,2013-020
2,2013-03-01,0.0,0.0,1489609.0,1464733.0,-1464733.0,0,2013-030
3,2013-04-01,0.0,0.0,1441550.0,1190657.0,-1190657.0,0,2013-040
4,2013-05-01,5.0,40.0,1212193.0,865478.0,-865438.0,0,2013-050
...,...,...,...,...,...,...,...,...
3745,2014-02-01,26.0,78.0,0.0,0.0,78.0,36,2014-0236
3746,2014-03-01,31.0,85.0,0.0,0.0,85.0,36,2014-0336
3747,2014-04-01,20.0,98.0,0.0,0.0,98.0,36,2014-0436
3748,2014-06-01,82.0,304.0,0.0,0.0,304.0,36,2014-0636


In [ ]:
12*37*4

1776

In [ ]:
pummok.head(5)

,datadate,단가(원),거래량,해당일자_전체평균가격(원),번호
0,2013-01-01,0.000000,0.000000,0.000000,0
1,2013-01-02,16802.716667,7.333333,20712.240909,0
2,2013-01-03,4454.533636,49.181818,3401.863678,0
3,2013-01-04,5287.541667,46.500000,6553.903226,0
4,2013-01-05,6308.730000,72.529412,4299.079157,0


In [ ]:
weather['품목'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36])

In [ ]:
imexport.info

,datadate,수출중량,수출금액,수입중량,수입금액,무역수지,num
0,2013-01-01,0.0,0.0,1864728.0,1195211.0,-1195211.0,0
1,2013-02-01,80.0,695.0,1285922.0,999313.0,-998618.0,0
2,2013-03-01,0.0,0.0,1489609.0,1464733.0,-1464733.0,0
3,2013-04-01,0.0,0.0,1441550.0,1190657.0,-1190657.0,0
4,2013-05-01,5.0,40.0,1212193.0,865478.0,-865438.0,0
...,...,...,...,...,...,...,...
3745,2014-02-01,26.0,78.0,0.0,0.0,78.0,36
3746,2014-03-01,31.0,85.0,0.0,0.0,85.0,36
3747,2014-04-01,20.0,98.0,0.0,0.0,98.0,36
3748,2014-06-01,82.0,304.0,0.0,0.0,304.0,36


In [ ]:
imexport.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3750 entries, 0 to 3749
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   datadate  3750 non-null   object 
 1   수출중량      3750 non-null   float64
 2   수출금액      3750 non-null   float64
 3   수입중량      3750 non-null   float64
 4   수입금액      3750 non-null   float64
 5   무역수지      3750 non-null   float64
 6   num       3750 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 205.2+ KB


In [ ]:
pummok.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54057 entries, 0 to 54056
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   datadate        54057 non-null  object 
 1   단가(원)           54057 non-null  float64
 2   거래량             54057 non-null  float64
 3   해당일자_전체평균가격(원)  54057 non-null  float64
 4   번호              54057 non-null  int64  
dtypes: float64(3), int64(1), object(1)
memory usage: 2.1+ MB


In [ ]:
pummok.tail(100)

,datadate,단가(원),거래량,해당일자_전체평균가격(원),번호
53957,2016-09-23,2661.977481,974.557252,2481.003447,36
53958,2016-09-24,2710.358140,1007.728682,2544.922604,36
53959,2016-09-25,2838.092353,958.669118,2598.258183,36
53960,2016-09-26,2899.836767,977.789474,2617.261470,36
53961,2016-09-27,2497.500000,1250.000000,2574.700800,36
...,...,...,...,...,...
54052,2016-12-27,0.000000,0.000000,0.000000,36
54053,2016-12-28,2753.411286,1009.285714,2428.960251,36
54054,2016-12-29,2799.843465,993.881890,2512.875229,36
54055,2016-12-30,3053.932411,1039.475177,2665.738013,36
